### Imports

In [101]:
import cv2 as cv
import numpy as np
import os
import time

In [102]:
def show_image(title, image):
    image = cv.resize(image, (0, 0), fx=0.2, fy=0.2)
    cv.imshow(title, image)
    cv.waitKey(0)
    cv.destroyAllWindows()

### Functia de detectie a colturilor patratului mare din tabla

In [103]:
def detectare_colturi(contours, img_final):
    gasit = False
        # print(s_min)
    for cont in contours:
        area = cv.contourArea(cont)

        # print(area)
        if 160000 < area < 170000:

            top_left = None
            bottom_right = None
            for point in cont.squeeze():
                if top_left is None or point[0] + point[1] < top_left[0] + top_left[1]:
                    top_left = point

                if bottom_right is None or point[0] + point[1] > bottom_right[0] + bottom_right[1] :
                    bottom_right = point

            diff = np.diff(cont.squeeze(), axis = 1)
            top_right = cont.squeeze()[np.argmin(diff)]
            bottom_left = cont.squeeze()[np.argmax(diff)]

            gasit = True
            # peri = cv.arcLength(cont, True)
            # cv.drawContours(img_final, cont, -1, (0, 100, 0), 3)
            # aprox = cv.approxPolyDP(cont, 0.01 * peri, True)
            # x, y, w, h = cv.boundingRect(aprox)
    # cv.imshow("colturi", img_final)
    return gasit, top_left, top_right, bottom_left, bottom_right, img_final


### Functia de cropare a patratului

In [104]:
def crop_img(img_final, top_left, top_right, bottom_left, bottom_right):
    width = 1800
    height = 1950

    pts1 = np.float32([top_left, top_right, bottom_left, bottom_right])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
    matrix = cv.getPerspectiveTransform(pts1, pts2)

    img_crop = cv.warpPerspective(img_final, matrix, (width, height))


    linii_verticale=[]
    for i in range(0,height + 1,120):
        l=[]
        l.append((i,0))
        l.append((i,height - 1))
        linii_verticale.append(l)

    linii_orizontale=[]
    for i in range(0,height + 1,130):
        l=[]
        l.append((0,i))
        l.append((width - 1,i))
        linii_orizontale.append(l)

    for line in  linii_verticale :
        cv.line(img_crop, line[0], line[1], (0, 255, 0), 5)
    for line in  linii_orizontale :
        cv.line(img_crop, line[0], line[1], (0, 0, 255), 5)
    img_crop = cv.resize(img_crop, (0, 0), fx=0.4, fy=0.4)
    cv.imshow("img cropata si impartita in linii", img_crop)

    return img_crop

### Functii de modificare a imaginii initiale cu operatii morfologice

In [105]:
### fereastra de proprietati custom
def empty(a):
    pass


def createGUI():
    cv.namedWindow("trackBars", cv.WINDOW_NORMAL)
    # cv.resizeWindow("trackBarks", 640, 240)
    cv.createTrackbar("Hue Min", "trackBars", 0, 179, empty)
    cv.createTrackbar("Hue Max", "trackBars", 179, 179, empty)
    cv.createTrackbar("Sat Min", "trackBars", 110, 255, empty)
    # intre 110 si 140
    cv.createTrackbar("Sat Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("Val Min", "trackBars", 0, 255, empty)
    cv.createTrackbar("Val Max", "trackBars", 255, 255, empty)
    cv.createTrackbar("MedBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("GausBlur", "trackBars", 1, 21, empty)
    cv.createTrackbar("Sharp alp", "trackBars", 12, 20, empty)
    cv.createTrackbar("Sharp bet", "trackBars", 8, 20, empty)
    cv.createTrackbar("Sharp gam", "trackBars", 0, 20, empty)
    cv.createTrackbar("tresmin", "trackBars", 5, 255, empty)
    cv.createTrackbar("tresmax", "trackBars", 255, 255, empty)
    cv.createTrackbar("cannymin", "trackBars", 0, 500, empty)
    cv.createTrackbar("cannymax", "trackBars", 0, 500, empty)
    cv.createTrackbar("erode", "trackBars", 3, 5, empty)


def track_bar_values():
    h_min = cv.getTrackbarPos("Hue Min", "trackBars")
    h_max = cv.getTrackbarPos("Hue Max", "trackBars")
    s_min = cv.getTrackbarPos("Sat Min", "trackBars")
    s_max = cv.getTrackbarPos("Sat Max", "trackBars")
    v_min = cv.getTrackbarPos("Val Min", "trackBars")
    v_max = cv.getTrackbarPos("Val Max", "trackBars")
    median = cv.getTrackbarPos("MedBlur", "trackBars")
    gaussian = cv.getTrackbarPos("GausBlur", "trackBars")
    alpha = cv.getTrackbarPos("Sharp alp", "trackBars")
    beta = cv.getTrackbarPos("Sharp bet", "trackBars")
    gama = cv.getTrackbarPos("Sharp gam", "trackBars")
    canny_min = cv.getTrackbarPos("cannymin", "trackBars")
    canny_max = cv.getTrackbarPos("cannymax", "trackBars")
    treshold_min = cv.getTrackbarPos("tresmin", "trackBars")
    treshold_max = cv.getTrackbarPos("tresmax", "trackBars")
    erode = cv.getTrackbarPos("erode", "trackBars")

    return h_min, h_max, s_min, s_max, v_min, v_max, median, gaussian, alpha, beta, gama, canny_min, canny_max, treshold_min, treshold_max, erode


In [106]:
def extrage_tabla(imgOriginala):

    h, w, _ = imgOriginala.shape
    # print(h, w)
    img = cv.resize(imgOriginala, (0, 0), fx=0.2, fy=0.2)

    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

    createGUI()
    h_min, h_max, s_min, s_max, v_min, v_max, median, gaussian, alpha, beta, gama, canny_min, canny_max, treshold_min, treshold_max, erode = track_bar_values()

    while True:
        # filtrare HSV
        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max, s_max, v_max])
        mask = cv.inRange(img_hsv, lower, upper)
        img_result = cv.bitwise_and(img, img, mask=mask)

        # HSV to GRAY
        h, s, v = cv.split(img_result)
        img_result = v

        # filtre morfologice
        img_m_blur = cv.medianBlur(img_result, median)
        img_g_blur = cv.GaussianBlur(img_result, (7, 7), gaussian)
        img_sharpened = cv.addWeighted(img_m_blur, alpha / 10, img_g_blur, (-1) * beta / 10, gama)
        _, thresh = cv.threshold(img_sharpened, treshold_min, treshold_max, cv.THRESH_BINARY)
        kernel = np.ones((erode, erode), np.uint8)
        thresh = cv.erode(thresh, kernel)
        edges = cv.Canny(thresh, canny_min, canny_max)

        # functia de detectare a contururilor
        contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        img_final = img.copy()
        # print(len(contours))

        # x, y pozitia stanga sus
        # w, h weight, height a patratului
        gasit, top_left, top_right, bottom_left, bottom_right, img_final = detectare_colturi(contours, img_final)

        # Daca gasim aria unui contur care se potriveste cu tabla noastra, se opreste din cautat, daca nu, marim saturatia pentru a rezolva problemele cu lumina
        if gasit:
            # cv.rectangle(img_final, tuple(top_left), tuple(bottom_right), (255, 255, 0), 2)
            pass
        else:
            s_min += 2
            if s_min > 150:
                break
            continue

        ### imagini cu rezultatul fiecarei operatii
        # cv.imshow("Thresh", thresh)
        # cv.imshow("Edges", edges)
        # cv.imshow("Original", img)
        # cv.imshow("BlurG", img_g_blur)
        # cv.imshow("BlurM", img_m_blur)
        # cv.imshow("sharpened", img_sharpened)
        # cv.imshow("Result", img_result)
        ### rezultatul cu patratul final
        # cv.circle(img_final,tuple(top_left),10,(0,0,255),-1)
        # cv.circle(img_final,tuple(top_right),10,(0,0,255),-1)
        # cv.circle(img_final,tuple(bottom_left),10,(0,0,255),-1)
        # cv.circle(img_final,tuple(bottom_right),10,(0,0,255),-1)
        cv.imshow("AFISARE PATRAT", img_final)




        img_final = crop_img(img_final, top_left, top_right, bottom_left, bottom_right)




        # apasa p pentru a trece la urmatoarea poza
        if cv.waitKey(0) & 0xFF == ord('p'):
            break

        # trece automat la urmatoarea poza dupa 0.3s
        # cv.waitKey(1)
        # time.sleep(0.3)
        # break

    cv.destroyAllWindows()

In [107]:
### 15 H
path = 'antrenare/'
img = cv.imread(path + "/1_01.jpg")

# imgModify(img)
files = os.listdir(path)
# print(files)
for file in files:
    if file[len(file) - 3:len(file)] == "jpg":
        print(file)
        img = cv.imread(path + file)
        extrage_tabla(img)

1_01.jpg


KeyboardInterrupt: 